In [1]:
from neo4j import GraphDatabase

def get_neo4j_driver() -> GraphDatabase.driver:
    """Establishes and returns a Neo4j session for AuraDB."""
    neo4j_uri = 'neo4j+s://9d1381c2.databases.neo4j.io:7687'
    neo4j_user = 'Shirley'
    neo4j_password = 'Sxl19950312'
    return GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

# Call the function to get the driver
driver = get_neo4j_driver()

# Open a session
with driver.session() as session:
    # Verify connectivity within the session
    session.run("RETURN 1")

print("Connectivity verified successfully.")


from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd


# Define a Cypher query to retrieve the required data
query = """
MATCH (n)
RETURN n
"""

# Execute the query and process the results
with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]

# Converting Neo4j Data to Pandas DataFrame Format
df = pd.DataFrame(neo4j_data)

#print(df)

Connectivity verified successfully.


In [3]:
import plotly.graph_objects as go
from neo4j import GraphDatabase


query = """
MATCH (ir:Allergen)
RETURN ir.severity AS Severity , ir.name AS AllergenName, ir.type AS AllergenType
"""


with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]

# Get a list of names and substitutions
names = [record['Severity'] for record in neo4j_data]
alternatives = [record['AllergenName'] for record in neo4j_data]
alternativess = [record['AllergenType'] for record in neo4j_data]

# Cteate chart
fig = go.Figure(data=[go.Table(
    header=dict(values=['Serverity', 'AllergenName','AllergenType'],
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[names, alternatives,alternativess],
               fill_color='lavender',
               align='left'))
])

# Updated Chart Layout
fig.update_layout(title='Allergen')

fig.show()

In [10]:
import networkx as nx

query = """ 
MATCH (ir:ManufacturingProcess)
RETURN id(ir) AS ManufacturingProcessId
"""


G = nx.Graph()


for mp_id in manufacturing_process_ids:
    G.add_node(mp_id)
   
    for related_id in get_related_mp_ids(mp_id):
        G.add_edge(mp_id, related_id)


communities = nx.community.girvan_newman(G)


for community in next(communities):
    subgraph = G.subgraph(community)
    conductance = nx.conductance(G, community)
    print(f"Community {community}: Conductance = {conductance:.4f}")

NameError: name 'get_related_mp_ids' is not defined

In [2]:
from neo4j import GraphDatabase

with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]

with driver.session() as session:
   
    graph = session.write_transaction(
        lambda tx: tx.run("""
            CALL gds.graph.project(
                'productGraph',
                'Product',
                {
                    SIMILAR_INGREDIENTS: {
                        orientation: 'UNDIRECTED'
                    },
                    SIMILAR_MANUFACTURING: {
                        orientation: 'UNDIRECTED'
                    }
                },
                {
                    nodeProperties: 'category',
                    relationshipProperties: {
                        SIMILAR_INGREDIENTS: 'ingredientScore',
                        SIMILAR_MANUFACTURING: 'processScore'
                    }
                }
            )
        """).data()
    )

 
    result = session.write_transaction(
        lambda tx: tx.run("""
            CALL gds.louvain.stream('productGraph')
            YIELD nodeId, componentId
            RETURN gds.util.asNode(nodeId).name AS product,
                   componentId AS communityId
            ORDER BY communityId
        """)
    )

    
    for record in result:
        print(f"Product: {record['product']}, Community: {record['communityId']}")

C:\Users\apple\AppData\Local\Temp\ipykernel_3244\1414059848.py:9: DeprecationWarning: write_transaction has been renamed to execute_write
  graph = session.write_transaction(


ClientError: {code: Neo.ClientError.Procedure.ProcedureNotFound} {message: There is no procedure with the name `gds.graph.project` registered for this database instance. Please ensure you've spelled the procedure name correctly and that the procedure is properly deployed.}

In [3]:
query = """ 
CALL gds.louvain.stream('myGraph')
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).name AS name, communityId
ORDER BY name ASC
""" 

In [ ]:
from neo4j import GraphDatabase



with driver.session() as session:
    
    graph = session.write_transaction(
        lambda tx: tx.run("""
            CALL gds.graph.project(
                'productGraph',
                ['Product', 'Recipe', 'Ingredient', 'ManufacturingProcess'],
                {
                    HAS_RECIPE: {
                        orientation: 'UNDIRECTED'
                    },
                    USES_INGREDIENT: {
                        orientation: 'UNDIRECTED'
                    },
                    HAS_MANUFACTURINGPROCESS: {
                        orientation: 'UNDIRECTED'
                    }
                },
                {
                    nodeProperties: 'category'
                }
            )
        """).data()
    )

    # Run Louvain's algorithm.
    result = session.write_transaction(
        lambda tx: tx.run("""
            CALL gds.louvain.stream('productGraph')
            YIELD nodeId, componentId
            RETURN gds.util.asNode(nodeId).name AS node,
                   labels(gds.util.asNode(nodeId)) AS labels,
                   componentId AS communityId
            ORDER BY communityId
        """)
    )

 
    for record in result:
        print(f"Node: {record['node']} ({','.join(record['labels'])}), Community: {record['communityId']}")

In [4]:
from graphdatascience import GraphDataScience
import pandas as pd
import os

ModuleNotFoundError: No module named 'graphdatascience'